In [ ]:
# IMPORTING DEPENDENCIES
import pandas as pd
import pandas_ta as pta
import talib as ta

# LOAD DATA
df=pd.read_csv(r"C:\Users\USER\Documents\New folder\BTC\BTCUSD_M15.csv",
                 delimiter='\t')
df['Time'] = pd.to_datetime(df['Time'])
df.set_index("Time", inplace=True)
# df = df.loc['2022-02-04 00:00:00':'2022-01-03 00:00:00']
df.info()

# VISUALIZING CLOSE PRICES
df['Close'].plot(figsize=(17,8))

# DEFINING NECESSARY TECHNICALS
df["VWAP"]=pta.vwap(df.High, df.Low, df.Close, df.Volume)
df["EMA"]=pta.ema(df.Close, length=200)
# CONSIDERING CANDLESTICK PATTERNS TO STREAMLINE SIGNALS
# df['Engulf'] = ta.CDLENGULFING(df.Open,df.High,df.Low,df.Close)

# EMA SIGNAL LOGIC
emasignal = [0]*len(df)
backcandles = 6
for row in range(backcandles, len(df)):
    upt = 1
    dnt = 1
    for i in range(row-backcandles, row+1):
        if df.High[i]>=df.EMA[i]:
            dnt=0
        if df.Low[i]<=df.EMA[i]:
            upt=0
    if upt==1 and dnt==1:
        emasignal[row]=3
    elif upt==1:
        emasignal[row]=2
    elif dnt==1:
        emasignal[row]=1

df['EMASignal'] = emasignal

# DEFINING THE VOLUME WEIGHTED PRICES
VWAPsignal = [0]*len(df)
backcandles = 5
for row in range(backcandles, len(df)):
    upt = 1
    dnt = 1
    for i in range(row-backcandles, row+1):
        if df.High[i]>=df.VWAP[i]:
            dnt=0
        if df.Low[i]<=df.VWAP[i]:
            upt=0
    if upt==1 and dnt==1:
        VWAPsignal[row]=3
    elif upt==1:
        VWAPsignal[row]=2
    elif dnt==1:
        VWAPsignal[row]=1

df['VWAPSignal'] = VWAPsignal

# TOTAL SIGNALS
def TotalSignal(l):
    myclosedistance = 100
    if (df.EMASignal[l]==2 and df.VWAPSignal[l]==2 #and df.Engulf[l]==2 
        and min(abs(df.VWAP[l]-df.High[l]),abs(df.VWAP[l]-df.Low[l]))<=myclosedistance):
            return 2
    if (df.EMASignal[l]==1 and df.VWAPSignal[l]==1 #and df.Engulf[l]==1 
        and min(abs(df.VWAP[l]-df.High[l]),abs(df.VWAP[l]-df.Low[l]))<=myclosedistance):
            return 1
        
TotSignal = [0]*len(df)
for row in range(0, len(df)):
    TotSignal[row] = TotalSignal(row)
df['TotalSignal'] = TotSignal